In [11]:
import oandapyV20.endpoints.instruments as instruments
import oandapyV20.endpoints.pricing as pricing
import oandapyV20.endpoints.orders as orders
import oandapyV20.endpoints.trades as trades
import pandas as pd
import oandapyV20 

In [12]:

# Account information
OANDA_ACCESS_TOKEN = "987464a00c47a9b186fcc7a93a9404a6-bf2ecb97ae681e4edeb529adef404b09"
ACCOUNT_ID = "101-001-8028197-001"

access_token = OANDA_ACCESS_TOKEN
accountID = ACCOUNT_ID

client = oandapyV20.API(access_token=access_token)

In [13]:
# Instrument (USDJPY)
instrument = "USD_JPY"

# Granularity (daily)
granularity = "D"

# Moving average periods
short_ma_period = 2
long_ma_period = 10

# Risk management
stop_loss_pips = 30
take_profit_pips = 60


In [19]:


# OANDA API connection setup
api = API(access_token=API_KEY, environment="practice")

# Function to fetch historical data
def get_historical_data(instrument, granularity, count):
    params = {
        "count": count,
        "granularity": granularity,
    }
    request = instruments.InstrumentsCandles(instrument=instrument, params=params)
    response = api.request(request)
    data = response['candles']
    ohlc_data = [{'time': candle['time'], 'open': float(candle['mid']['o']), 'high': float(candle['mid']['h']),
                  'low': float(candle['mid']['l']), 'close': float(candle['mid']['c'])} for candle in data]
    df = pd.DataFrame(ohlc_data)
    df['time'] = pd.to_datetime(df['time'])
    df.set_index('time', inplace=True)
    return df

# Define the instrument, granularity, and count
instrument = 'USD_JPY'
granularity = 'D'  # Daily data
count = 5000  # Maximum number of candles to retrieve

# Fetch historical data
historical_data = get_historical_data(instrument, granularity, count)

# Print the fetched data
print(historical_data)


                              open     high      low    close
time                                                         
2006-11-16 22:00:00+00:00  118.220  118.462  117.521  117.755
2006-11-18 22:00:00+00:00  117.775  117.970  117.650  117.865
2006-11-19 22:00:00+00:00  117.860  118.192  117.795  118.047
2006-11-20 22:00:00+00:00  118.057  118.255  117.751  117.904
2006-11-21 22:00:00+00:00  117.894  117.915  116.375  116.751
...                            ...      ...      ...      ...
2024-01-29 22:00:00+00:00  147.415  147.928  147.098  147.609
2024-01-30 22:00:00+00:00  147.587  147.902  146.012  146.965
2024-01-31 22:00:00+00:00  146.978  147.118  145.898  146.418
2024-02-01 22:00:00+00:00  146.401  148.587  146.243  148.382
2024-02-04 22:00:00+00:00  148.431  148.894  148.268  148.624

[5000 rows x 4 columns]
